In [ ]:
%reload_ext autoreload

Error: Session cannot generate requests

In [2]:
import copy
from tqdm import tqdm
import numpy as np

In [3]:
import sys
print(sys.executable)

/home/hazem/anaconda3/envs/anaconda_enviroment/bin/python


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [5]:
%load_ext autoreload
%autoreload 2
from models import ResNet18
from client import Client
from data import medmnist_dataset

In [6]:
def average_weights(w):
    """
    Returns the average of the wei'ghts.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg


In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLIENTS = 100
EPOCHS = 10
FRACTION = 0.2
EVERY = 1
LOCAL_EPOCHS = 10
DATA_POINTS_PER_USER = 100
print(type(medmnist_dataset))
train_dataset, test_dataset, clients, info = medmnist_dataset(name="PathMnist", num_clients=NUM_CLIENTS, DATA_POINTS_PER_USER)
n_channels = info['n_channels']
n_classes = len(info['label'])

<class 'function'>
Using downloaded and verified file: ../../data/pathmnist/pathmnist.npz
Using downloaded and verified file: ../../data/pathmnist/pathmnist.npz
Using downloaded and verified file: ../../data/pathmnist/pathmnist.npz


In [24]:
print(len(clients[list(clients.keys())[0]]))

899


In [8]:
print(test_dataset)

Dataset PathMNIST (pathmnist)
    Number of datapoints: 7180
    Root location: ../../data/pathmnist
    Split: test
    Task: multi-class
    Number of channels: 3
    Meaning of labels: {'0': 'adipose', '1': 'background', '2': 'debris', '3': 'lymphocytes', '4': 'mucus', '5': 'smooth muscle', '6': 'normal colon mucosa', '7': 'cancer-associated stroma', '8': 'colorectal adenocarcinoma epithelium'}
    Number of samples: {'train': 89996, 'val': 10004, 'test': 7180}
    Description: The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (NCT-CRC-HE-100K) of 100,000 non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (CRC-VAL-HE-7K) of 7,180 image patches from a different clinical center. The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task. We resize the source images of 3×224×224 into 3×28×28, and split NCT-CRC-HE-100K into tr

In [19]:
print(info["task"])

multi-class


In [9]:
global_model = ResNet18(in_channels=n_channels, num_classes=n_classes)
global_model.to(DEVICE)
global_model.train()
local_weights = []
local_losses = []
train_loss = []
training_accuracy = []

In [10]:
print(len(clients[0]))

899


In [13]:
lr = 0.001
for epoch in tqdm(range(EPOCHS)):
    local_weights, local_losses = [], []
    global_model.train()
    m = max(int(FRACTION*NUM_CLIENTS), 1) # number of clients for each round
    global_weights = global_model.state_dict()
    
    set_of_clients = np.random.choice(range(NUM_CLIENTS), m, replace=False)
    for client_idx in tqdm(set_of_clients):
        print(f"client {client_idx}")
        local_model = Client(dataset=train_dataset, idxs=list(clients[client_idx]),local_epochs=10, local_bs=10, device=DEVICE)
        print(f"updating weights for client {client_idx}")
        weights , loss = local_model.update_weights(model=copy.deepcopy(global_model), global_round=epoch, lr=lr)
        local_weights.append(copy.deepcopy(weights)) 
        local_losses.append(copy.deepcopy(loss))
    
    # get the averaged weights
    print(f'Averaging the model for epoch number {epoch}')
    global_weights = average_weights(local_weights)
    
    # update global model with averaged weights
    global_model.load_state_dict(global_weights)

    # average of the losses
    loss_avg = np.mean(local_losses)
    train_loss.append(loss_avg)

    # getting the average training accuracy over all clients at every epoch
    print(f"getting the average training accuracy over all clients at epoch number {epoch}")
    list_acc, list_loss = [], []
    global_model.eval()
    for client in range(NUM_CLIENTS):
        local_model = Client(dataset=train_dataset, idxs=list(clients[client_idx]),local_epochs=10, local_bs=10, device=DEVICE)
        acc, loss = local_model.test_model(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    training_accuracy.append(np.mean(list_acc))

    if epoch % EVERY == 0:
        print(f"EPOCH {epoch + 1}:")
        print(f'Average training loss: {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*training_accuracy[-1]))
    
# Testing how the model is doing after all the traingin rounds
global_model.eval()
loss, total, correct = 0.0, 0.0, 0.0

criterion = nn.NLLLoss().to(DEVICE)
testloader = DataLoader(test_dataset, batch_size=128,
                        shuffle=False)

for batch_idx, (images, labels) in enumerate(testloader):
    images, labels = images.to(DEVICE), labels.to(DEVICE)

    # Inference
    outputs = global_model(images)
    batch_loss = criterion(outputs, labels)
    loss += batch_loss.item()

    # Prediction
    _, pred_labels = torch.max(outputs, 1)
    pred_labels = pred_labels.view(-1)
    correct += torch.sum(torch.eq(pred_labels, labels)).item()
    total += len(labels)

accuracy = correct/total
print(f' \n Results after {EPOCHS} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*training_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*accuracy))

  0%|          | 0/10 [00:00<?, ?it/s]/home/hazem/Basira-Internship/code/FederatedLearningPytorchHazem/src/client.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


client 34
updating weights for client 34


  0%|          | 0/10 [00:16<?, ?it/s]


KeyboardInterrupt: 